Objectif : merger les datasets de resultats world cup.

In [2]:
import pandas as pd
import os

WC_RESULTS_DIR = "wc2018_results_data"

# Chargement du CSV : Goals
CSV_PATH_GOALS = os.path.join(WC_RESULTS_DIR, "World_cup_2018_goals.csv")
df_goals = pd.read_csv(CSV_PATH_GOALS)

# Chargement du CSV : Country
CSV_PATH_COUNTRY = os.path.join(WC_RESULTS_DIR, "World_cup_2018_country.csv")
df_country = pd.read_csv(CSV_PATH_COUNTRY)

# Chargement du CSV : Matches
CSV_PATH_MATCHES = os.path.join(WC_RESULTS_DIR, "World_cup_2018_matches.csv")
df_matches = pd.read_csv(CSV_PATH_MATCHES)

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("WorldCup_Final_Merge").getOrCreate()

# Chargement
df_goals = spark.read.option("header", "true").csv("goals.csv")
df_countries = spark.read.option("header", "true").csv("countries.csv")
df_results = spark.read.option("header", "true").csv("results.csv")

# --- ÉTAPE 1 : Préparer les buts ---
# On agrège les buts par match pour éviter de dupliquer les lignes de résultats
df_goals_agg = df_goals.groupBy("Date", "Home", "Away").agg(
    F.collect_list(F.struct("Player_scored", "Time", "Team_scored")).alias("goals_detail")
)

# --- ÉTAPE 2 : Joindre Résultats et Buts ---
# On garde tous les matchs (même les 0-0) avec un Left Join
df_merged = df_results.join(df_goals_agg, on=["Date", "Home", "Away"], how="left")

# --- ÉTAPE 3 : Enrichir avec les infos Pays ---
# On ajoute les infos pour l'équipe à domicile (Home)
df_final = df_merged.join(
    df_countries.select("Country", "World_ranking", "Group").withColumnRenamed("Country", "Home"),
    on="Home",
    how="left"
).withColumnRenamed("World_ranking", "Home_Rank").withColumnRenamed("Group", "Home_Group")

# On ajoute les infos pour l'équipe à l'extérieur (Away)
df_final = df_final.join(
    df_countries.select("Country", "World_ranking").withColumnRenamed("Country", "Away"),
    on="Away",
    how="left"
).withColumnRenamed("World_ranking", "Away_Rank")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/06 11:42:01 WARN Utils: Your hostname, MacBook-Air-de-Leo.local, resolves to a loopback address: 127.0.0.1; using 192.168.166.130 instead (on interface en0)
26/02/06 11:42:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/06 11:42:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/06 11:42:03 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: goals.csv.
java.io.FileNotFoundException: File goals.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:980)
	at org.apache.hadoo

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/leoalb/Documents/MS data/Cours/UE10 Information retrieval/Babiga/World-cup-tweets-analysis/goals.csv. SQLSTATE: 42K03